In [1]:
from subtitle_process import fetch_subtitle
import getpass
import os
import json
import re
from templates import *
from functions import inference

In [2]:
# 设置临时环境变量
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("bilibili_ssid")

In [3]:
with open("url.json", 'r') as file:
    data = json.load(file)

In [4]:
data[0]

'BV1iT411q7GM'

In [5]:
problems = []
for bv in data:
    problems.append(fetch_subtitle(bv))

Extracted segment: 1iT411q7GM
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/484220007107407896653ecb5f2122578ac2f3848cad97b7709?auth_key=1720582925-56ca178565ec4dd5b091ade3d0433c2f-0-433e276f10127a3e3e9c499892631fc3


In [6]:
def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)\n\n", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
            })

    with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


In [9]:
template_sub_1_similar = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一个相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

视频字幕：
{subtitle}

注意事项：
1. 对于每一道原题，仅构造一道相似题，对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话，每一个讲解必须以答案结尾，不能够没有答案。
2. 保留老师讲解风格但是把和讲解无关的内容过滤掉，比如自问自答的部分都要删除掉。
"""

template_expl = r"""
    Role:
        你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。 
    Goals:
        根据给定的板书内容像老师一样讲解板书内容，根据输入决定你正在输出的是讲解几，比如如果输入是板书一，那么输出就是讲解一，以此类推。

    严格按照以下列字符串格式输出:
        ### 讲解一:    ### 讲解二:      ...

    原题和参考信息:
        题目:{question}
        参考信息:{ref}

    注意事项:
        1. 你的讲解要尽量贴合人类老师的讲解风格，言简意赅，清晰明了。
        2. 讲解内容应针对板书的具体步骤进行详细说明，解释每一步的思路和方法。
        3. 确保每一个讲解都对应一个板书。
        4. 板书和讲解的编号必须一一对应，格式必须严格遵守。
        5. 讲解内容应突出解题思路和方法，避免过多冗余信息。
        6. 你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇
        7. 其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题，别用括号1234。
            d: 初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
            e: 对于所有题目尽可能有最简单的方式来进行解答，以最大限度上避免超纲的情况发生。

    下面是板书的内容(请确保每一个讲解都对应一个板书)：
    {notes}

    下面是一些名师讲解的其他题目，请你学习他们的讲解风格，并运用到板书的讲解中(注意：你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇)。
    注意：初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
    {model_expl}
"""

In [10]:
raw_problem_answer = []
for problem in problems:
    raw_problem_answer.append(await inference({"subtitle": problem}, template_sub_1_similar))


executing
### 原题：
两个数相减的差是7.2。如果被减数增加8.6，减数减少2.7，则现在的差是多少？

### 讲解：
首先，我们要理解题目中的关键变化。被减数增加8.6，那么差也会增加8.6。减数减少2.7，那么差也会增加2.7。我们可以将这些变化逐步加到原来的差上。

原来的差是7.2。被减数增加8.6，差也增加8.6，所以新的差是7.2 + 8.6。然后，减数减少2.7，差也增加2.7，所以新的差是7.2 + 8.6 + 2.7。

我们来计算一下：
7.2 + 8.6 = 15.8
15.8 + 2.7 = 18.5

所以，现在的差是18.5。

### 相似题一：
两个数相减的差是5.4。如果被减数增加6.3，减数减少1.8，则现在的差是多少？

### 相似题一讲解：
首先，我们要理解题目中的关键变化。被减数增加6.3，那么差也会增加6.3。减数减少1.8，那么差也会增加1.8。我们可以将这些变化逐步加到原来的差上。

原来的差是5.4。被减数增加6.3，差也增加6.3，所以新的差是5.4 + 6.3。然后，减数减少1.8，差也增加1.8，所以新的差是5.4 + 6.3 + 1.8。

我们来计算一下：
5.4 + 6.3 = 11.7
11.7 + 1.8 = 13.5

所以，现在的差是13.5。

In [11]:
responses = raw_problem_answer
questions_and_explanations = []
for response in responses:
    print(type(response))
    pattern = re.compile(r"### 原题：\n(.*?)\n\n### 讲解：\n(.*?)\n\n### 相似题一：\n(.*?)\n\n### 相似题一讲解：\n(.*?)\n\n", re.DOTALL)
    matches = pattern.findall(response)
    print(matches)
    for match in matches:
        questions_and_explanations.append({
            "original_question": match[0].strip(),
            "original_explanation": match[1].strip(),
            "similar_question_1": match[2].strip(),
            "similar_explanation_1": match[3].strip(),
        })

with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
    json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


<class 'str'>
[('两个数相减的差是7.2。如果被减数增加8.6，减数减少2.7，则现在的差是多少？', '首先，我们要理解题目中的关键变化。被减数增加8.6，那么差也会增加8.6。减数减少2.7，那么差也会增加2.7。我们可以将这些变化逐步加到原来的差上。\n\n原来的差是7.2。被减数增加8.6，差也增加8.6，所以新的差是7.2 + 8.6。然后，减数减少2.7，差也增加2.7，所以新的差是7.2 + 8.6 + 2.7。\n\n我们来计算一下：\n7.2 + 8.6 = 15.8\n15.8 + 2.7 = 18.5\n\n所以，现在的差是18.5。', '两个数相减的差是5.4。如果被减数增加6.3，减数减少1.8，则现在的差是多少？', '首先，我们要理解题目中的关键变化。被减数增加6.3，那么差也会增加6.3。减数减少1.8，那么差也会增加1.8。我们可以将这些变化逐步加到原来的差上。')]


In [12]:
extract_question_explanations(raw_problem_answer)

In [13]:
with open("casestudy_problem_answer.json", 'r') as file:
    data = json.load(file)

data

[{'original_question': '两个数相减的差是7.2。如果被减数增加8.6，减数减少2.7，则现在的差是多少？',
  'original_explanation': '首先，我们要理解题目中的关键变化。被减数增加8.6，那么差也会增加8.6。减数减少2.7，那么差也会增加2.7。我们可以将这些变化逐步加到原来的差上。\n\n原来的差是7.2。被减数增加8.6，差也增加8.6，所以新的差是7.2 + 8.6。然后，减数减少2.7，差也增加2.7，所以新的差是7.2 + 8.6 + 2.7。\n\n我们来计算一下：\n7.2 + 8.6 = 15.8\n15.8 + 2.7 = 18.5\n\n所以，现在的差是18.5。',
  'similar_question_1': '两个数相减的差是5.4。如果被减数增加6.3，减数减少1.8，则现在的差是多少？',
  'similar_explanation_1': '首先，我们要理解题目中的关键变化。被减数增加6.3，那么差也会增加6.3。减数减少1.8，那么差也会增加1.8。我们可以将这些变化逐步加到原来的差上。'}]

In [14]:
model_explanation = ""
for ps in data[:3]:
    model_explanation += "题目:\n" + ps['original_question'] + '\n\n'
    model_explanation += "讲解:\n" + ps['original_explanation'] + '\n\n'
    # model_explanation += "题目:\n" + ps['similar_question_1'] + '\n\n'
    # model_explanation += "讲解:\n" + ps['similar_explanation_1'] + '\n\n'
model_explanation

'题目:\n两个数相减的差是7.2。如果被减数增加8.6，减数减少2.7，则现在的差是多少？\n\n讲解:\n首先，我们要理解题目中的关键变化。被减数增加8.6，那么差也会增加8.6。减数减少2.7，那么差也会增加2.7。我们可以将这些变化逐步加到原来的差上。\n\n原来的差是7.2。被减数增加8.6，差也增加8.6，所以新的差是7.2 + 8.6。然后，减数减少2.7，差也增加2.7，所以新的差是7.2 + 8.6 + 2.7。\n\n我们来计算一下：\n7.2 + 8.6 = 15.8\n15.8 + 2.7 = 18.5\n\n所以，现在的差是18.5。\n\n'

In [15]:
orig_answers = []
orig_notes = []
orig_expls = []
refined_explanation = []
questions = []

In [16]:
'''
对于每一个<原题,相似题>，用两个prompt来分别对相似题进行解答
    1. 用original prompt对相似题直接进行解答
    2. 将原题解答及相似题解答作为few shots examples喂给另一个prompt来对比两种的效果哪一个比较好
'''

with open("casestudy_problem_answer.json", 'r') as file:
    problem_set = json.load(file)


for ps in problem_set:
    orig_answers.append(await inference({"question": ps['similar_question_1'], "ref_form": ref_form, "ref": ref}, template_orig))
    pattern1 = r"### 板书一:[\s\S]*?(?:### 板书二:[\s\S]*?)?(?=### 讲解一)"
    orig_note = re.search(pattern1, orig_answers[-1]).group()
    pattern2 = r"### 讲解一:[\s\S]*"
    orig_expl = re.search(pattern2, orig_answers[-1]).group()
    orig_notes.append(orig_note)
    orig_expls.append(orig_expl)
    questions.append(ps['similar_question_1'])
    refined_explanation.append(await inference({"question": ps['similar_question_1'], "ref": ref, "notes": orig_note, "model_expl": model_explanation}, template_expl))

# print(orig_notes)
# print(refined_explanation)    

executing
知识点讲解:本题考查的知识点是代数中的加减法，解题关键点是理解被减数和减数的变化对差的影响。

### 板书一:
设原被减数为$x$，原减数为$y$，
依题意得：
$x - y = 5.4$

### 板书二:
被减数增加6.3，减数减少1.8，
则新的被减数为$x + 6.3$，
新的减数为$y - 1.8$，
新的差为：
$(x + 6.3) - (y - 1.8)$

### 板书三:
$\because x - y = 5.4$
$\therefore (x + 6.3) - (y - 1.8) = (x - y) + 6.3 + 1.8$
$= 5.4 + 6.3 + 1.8$
$= 13.5$

故答案是：13.5

### 讲解一:
首先设原被减数为$x$，原减数为$y$，根据题意可以得到方程$x - y = 5.4$。

### 讲解二:
接着，题目给出被减数增加6.3，减数减少1.8，因此新的被减数为$x + 6.3$，新的减数为$y - 1.8$，新的差为$(x + 6.3) - (y - 1.8)$。

### 讲解三:
利用已知的$x - y = 5.4$，我们可以将新的差表示为$(x - y) + 6.3 + 1.8$，计算得出新的差为13.5。故答案是13.5。executing
### 讲解一:
首先，我们需要设定一些变量来表示题目中的被减数和减数。设原被减数为$x$，原减数为$y$。根据题目中的信息，我们知道两个数相减的差是5.4，因此可以写出以下等式：
\[ x - y = 5.4 \]
这一步是为了明确题目给出的初始条件。

### 讲解二:
接下来，我们根据题目中的变化来调整被减数和减数。题目说被减数增加6.3，减数减少1.8。因此，新的被减数可以表示为$x + 6.3$，新的减数可以表示为$y - 1.8$。于是，新的差可以表示为：
\[ (x + 6.3) - (y - 1.8) \]
这一步是为了将题目中的变化具体化，方便后续计算。

### 讲解三:
现在，我们将新的差展开并进行计算。首先，根据之前的等式$x - y = 5.4$，我们可以将新的差表示为：
\[ (x + 6.3) - (y - 1.8) = (x - y) + 6.3 + 1.8 \]
因为$x - y = 5.4$，所以可以代入得

In [18]:
with open("compare_results_2.md", 'a') as file:
    for i in range(len(questions)):
        file.write("# 原题：\n" + questions[i] + '\n\n')
        file.write("# 原本板书：\n" + orig_notes[i] + '\n\n')
        file.write("# 原本解答：\n" + orig_expls[i] + '\n\n')
        file.write("# 新解答：\n" + refined_explanation[i] + '\n\n')

In [31]:
print(len(orig_expls))

12


In [32]:
import re

text = """
知识点讲解:本题考查的知识点是因式分解与方程求解，解题关键点是利用平方差公式将原式化简并求解。

###板书一:
解：
$(2a+2b+1)(2a+2b-1)=63$

$设x=2a+2b$

$则(x+1)(x-1)=63$

$x^2-1=63$

$x^2=64$

$x=±8$

$2a+2b=8$

$a+b=4$

$2a+2b=-8$

$a+b=-4$

故答案是$a+b=4$或$a+b=-4$

###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。
"""

pattern = r"###讲解一:[\s\S]*"
match = re.search(pattern, text).group()

if match:
    print(match)
else:
    print("No match found")


###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。

